In [1]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import psycopg2
import os
%matplotlib inline
#plt.style.use('ggplot')
pd.set_option('display.max_columns', None)

In [8]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler,FunctionTransformer,StandardScaler,OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import StratifiedKFold,StratifiedShuffleSplit, train_test_split, learning_curve,GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score, roc_curve
from imblearn.over_sampling import RandomOverSampler
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
rnd_state = 0

In [3]:
# create a database connection
sqluser = 'maxim'
dbname = 'maxim'
schema_name = 'mimiciii'

In [4]:
# Connect to local postgres version of mimic
connect = psycopg2.connect(dbname=dbname, user=sqluser)
cursor = connect.cursor()
cursor.execute('SET search_path to {}'.format(schema_name))

In [6]:
# Load in the query from file
query='SELECT * FROM APSIII'
aps = pd.read_sql_query(query, connect)
aps = aps.dropna(axis=0)
#aps = aps.loc[(oasis.urineoutput < 20000) & (oasis.urineoutput > 0) & 
#                  (oasis.temp > 30) & (oasis.temp < 45) & (oasis.icustay_age_group == 'adult')]
#oasis['preiculos_hours'] = oasis.preiculos.dt.total_seconds()/3600

In [7]:
aps.head()

,subject_id,hadm_id,icustay_id,apsiii,apsiii_prob,hr_score,meanbp_score,temp_score,resprate_score,pao2_aado2_score,hematocrit_score,wbc_score,creatinine_score,uo_score,bun_score,sodium_score,albumin_score,bilirubin_score,glucose_score,acidbase_score,gcs_score
1,27513,163557,200003,48,0.102705,7.0,10.0,0.0,9.0,0.0,3.0,5.0,0.0,0.0,7.0,0.0,0.0,6.0,0.0,1.0,0.0
8,9514,127229,200014,50,0.111749,0.0,10.0,8.0,6.0,0.0,3.0,0.0,0.0,7.0,7.0,0.0,6.0,0.0,0.0,0.0,3.0
18,41710,181955,200028,53,0.126616,5.0,15.0,0.0,6.0,0.0,3.0,0.0,0.0,5.0,2.0,0.0,0.0,0.0,5.0,12.0,0.0
46,19891,112840,200065,121,0.782868,13.0,9.0,20.0,11.0,0.0,3.0,5.0,10.0,15.0,7.0,0.0,11.0,0.0,5.0,12.0,0.0
47,214,197273,200066,58,0.155130,5.0,7.0,2.0,6.0,2.0,3.0,1.0,7.0,7.0,11.0,0.0,0.0,0.0,0.0,4.0,3.0
